[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AethiQs/PTC_PA2/blob/main/PTCII_WEEK4_Exercises_Answers.ipynb)

# PTC Python Advanced II:

## Exercises week 4

In this Jupytern otebook you will practice with some of the concepts covered in the first three weeks. More specifically, you will ...

- load and preprocess a dataset
- make a train-test split of the data
- train a logistic regression model
- evaluate a model

## Working in Google Colab
For this course we will work in Google Colab. This is a free online Python environment that has many useful packages already installed (including sklearn!). We will prepare the notebooks so that you only need to write code and check the output, without the hassle of downloading datasets and installing packages.

If you haven't done so already, please click the `Open in Colab` button at the top of the notebook. This will open this notebook in the Colab interface. If necessary, you can log in with your school account. To editg the notebook, you must copy it to your personal drive. This is easily done by clicking the button "Kopiëren naar Drive" at the top of the Colab menu.

In [19]:
# Run this cell to check if the notebook is working properly
import sklearn
print("Sklearn version: ", sklearn.__version__)

# Enter your name here and run the notebook to check you can edit cells
MY_NAME = "EDIT ME"
print("My name is ", MY_NAME)

Sklearn version:  1.0.2
My name is  EDIT ME


### 1. In this question we load in the data for you. The data is loaded from a web url so that you don't need to connect Google Colab to your drive.

**DO NOT CHANGE THE CODE IN THE CELL BELOW PLEASE**

In [20]:
import pandas as pd

# DO NOT CHANGE THIS CODE
URL = "https://raw.githubusercontent.com/AethiQs/PTC_PA2/main/medical.csv"
medical_data = pd.read_csv(URL, sep=",") \
    .drop(['slope', 'ca', 'thal'], axis=1) \
    .replace("?", pd.NA) \
    .dropna() \
    .astype(float) \
    .reset_index(drop=True)

medical_data

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,num
0,28.0,1.0,2.0,130.0,132.0,0.0,2.0,185.0,0.0,0.0,0.0
1,29.0,1.0,2.0,120.0,243.0,0.0,0.0,160.0,0.0,0.0,0.0
2,30.0,0.0,1.0,170.0,237.0,0.0,1.0,170.0,0.0,0.0,0.0
3,31.0,0.0,2.0,100.0,219.0,0.0,1.0,150.0,0.0,0.0,0.0
4,32.0,0.0,2.0,105.0,198.0,0.0,0.0,165.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
256,52.0,1.0,4.0,160.0,331.0,0.0,0.0,94.0,1.0,2.5,1.0
257,54.0,0.0,3.0,130.0,294.0,0.0,1.0,100.0,1.0,0.0,1.0
258,56.0,1.0,4.0,155.0,342.0,1.0,0.0,150.0,1.0,3.0,1.0
259,58.0,0.0,2.0,180.0,393.0,0.0,0.0,110.0,1.0,1.0,1.0


### 3. The `num` column is the target column we want to predict using the values in the other columns. A value of 1 indicates that a patient suffers from a heart disease. Create two dataframes, named `X` and `y`. Make sure that `y` contains only the column `num` and that `X` contains all other columns except `num`. You can make use of simple column selection to make `y`, and the `.copy()` and `.drop()` methods to make `X`. Print both dataframes to make sure you did it correctly.

In [37]:
# The task of predicting whether `num` is 1 or 0 is often called:
# [X] Classification
# [ ] Regression
# [ ] Clustering
# Place an "X" at the correct answer

# Get columns names
print(medical_data.columns)


# Creating dataframes (note spaces after num column)
X = medical_data.drop(columns="num       ")
y = medical_data["num       "]


# Show dataframe (replace with y to check)
print(X)
print(y)

Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'num       '],
      dtype='object')
      age  sex   cp  trestbps   chol  fbs  restecg  thalach  exang  oldpeak
0    28.0  1.0  2.0     130.0  132.0  0.0      2.0    185.0    0.0      0.0
1    29.0  1.0  2.0     120.0  243.0  0.0      0.0    160.0    0.0      0.0
2    30.0  0.0  1.0     170.0  237.0  0.0      1.0    170.0    0.0      0.0
3    31.0  0.0  2.0     100.0  219.0  0.0      1.0    150.0    0.0      0.0
4    32.0  0.0  2.0     105.0  198.0  0.0      0.0    165.0    0.0      0.0
..    ...  ...  ...       ...    ...  ...      ...      ...    ...      ...
256  52.0  1.0  4.0     160.0  331.0  0.0      0.0     94.0    1.0      2.5
257  54.0  0.0  3.0     130.0  294.0  0.0      1.0    100.0    1.0      0.0
258  56.0  1.0  4.0     155.0  342.0  1.0      0.0    150.0    1.0      3.0
259  58.0  0.0  2.0     180.0  393.0  0.0      0.0    110.0    1.0      1.0
260  65.0  1.0  4.0     

### 4. It is important to set data aside for testing the model. We therefore create a train-test split of the data, where we keep 15% of the data as test data. We use stratification to make sure that the distribution of patients, i.e., the `num` column, is roughly the same in the train and test data. We provide this code for you here so that you can use it later and focus on the model training.

In [39]:
# Import
from sklearn.model_selection import train_test_split


# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.15,
    random_state=42,
    shuffle=True,
    stratify=y
)

### 5. Among the five categorical variables, only `sex`, `fbs`, and `exang` have two levels of 0 and 1, so they are already in the dummy variable format. We still need to convert `cp` and `restecg` into dummy variables however. Use an SKlearn transformer to create columns of dummy variables corresponding to the variables `cp` and `restecg`. Hint: you may want to use the SKlearn `OneHotEncoder`. Don't forget to import all (and only) the functions you want to use from SKlearn! Also make sure that you only fit the transformer on the training data!

In [40]:
# Import
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder


# We will use OneHotEncoder to transform the cp and restecg columns
# We only need to process these columns, so we use a ColumnTransformer
categorical_columns = ["cp", "restecg"]
preprocessor = ColumnTransformer(
    transformers=[
        ("dummy", OneHotEncoder(), categorical_columns),
    ],
    remainder="passthrough"
)


# Apply transformer to train and test data
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

### 6. In the previous questions you should have recognized that we are doing classification. In this question you will use a very simple classification model to see if you can predict whether `num` should be 0 or 1 for a patient based on the variables in `X`. First create a `LogisticRegression` transformer from SKlearn. Train this model on the **train** data using the `.fit()` method. We want you to figure the exact code out by yourself by looking at the example in the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html). Don't worry about the hyperparameters for now and just use the defaults!

In [42]:
# Import
from sklearn.linear_model import LogisticRegression

# Create model
model = LogisticRegression()


# Train model on training data
model.fit(X_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

### 7. Finally, we will evaluate the data on the testing data. To do this, we must first apply our model to `X_test` using the `.predict()` method. We then use the `classification_report` function in SKlearn to interpret the resutls by comparing the prediction with the original data (`y_test`). You can refer to the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) for the classification report function and code examples.

In [48]:
# Imports
from sklearn.metrics import classification_report

# Apply model to X_test
y_pred = model.predict(X_test)

# Run the classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.86      1.00      0.93        25
         1.0       1.00      0.73      0.85        15

    accuracy                           0.90        40
   macro avg       0.93      0.87      0.89        40
weighted avg       0.91      0.90      0.90        40



In [ ]:
# Based on the classification report baove, how do you think the model performs>
# Add your answers/findings below in a comment:
"""    
TYPE HERE
"""